작성일자: 2022-07-21 <br>
작성자: 김동연

In [1]:
from collections import defaultdict

In [2]:
names=['Socreates', 'Archimedes', "Plato", "Aristotle"]
names.sort(key=lambda x : len(x))
print(names)

['Plato', 'Socreates', 'Aristotle', 'Archimedes']


파이썬 API들 상당수는 함수를 parameter로 받아서 기능을 수행할 수 있다.<br>
이 parameter들을 hook이라고 한다.<br>
위의 코드에서는 람다 표현식을 parameter로 받아 리스트를 정렬한다.<br>
파이썬은 first-class function을 갖추었기 때문에 후크로 함수를 사용할 수 있다.

<br>
<br>
collections 모듈의 defaultdict 클래스는 딕셔너리에 key와 value를 넣을 때 value를 따로 지정해주지 않아도 고정값을 제공한다.<br>
첫번째 parameter로 hook을 제공하면 존재하지 않는 키에 대해 조회할 경우 hook의 return value를 value로 지정한다.

In [4]:
def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Before: ', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before:  {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


생각한 것처럼 log_missing이 두 번 실행된 것을 확인할 수 있다.<br>
<br>
<br>
만약 hook을 이용해서 찾을 수 없었던 key의 총 개수를 센다고 하면 어떤 방법을 사용할 수 있을까?<br>
가장 대표적으로 헬퍼 함수를 사용할 수 있을 것이다.

In [6]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

result, count = increment_with_report(current, increments)
assert count == 2

하지만 이 방법은 함수를 이해하기가 어렵다는 단점이 있다.<br>
위의 함수를 보고 단시간에 defaultdict의 key의 개수를 센다는 것을 알기는 쉽지 않다.<br>
클래스를 사용한다면 어떨까?

In [8]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0

counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount

assert counter.added == 2

더 명확하게 이해를 할 수 있다.<br>
하지만 클래스 코드만 보고는 이것이 무엇을 하기 위한 클래스인지는 이해하기 어렵다.<br>
이런 상황을 위해 python에는 ```__call__```이라는 매서드를 정의해두었다.<br>
이 메서드는 instance를 함수처럼 호출할 수 있게 한다.

In [ ]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0
    
counter = BetterCountMissing()
